In [1]:
from nn import QLearningAgent, QNetwork
from verify_data import FishGame, ParseError

In [2]:
import os

memories = []
for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)
    with open(filepath, 'r') as f:
        try:
            print(f"{filename}")
            game = FishGame(f.readlines())
            for player in game.players:
                memories += game.memory(player)
        except ParseError as e:
            print(f"{filename}: {e}")
            break

12-10_11:07.txt
12-3_14:05.txt
12-3_11:30.txt
12-3_14:27.txt
12-6_11:08.txt
12-3_11:12.txt
1-15_11:15.txt


In [3]:
# with open('data/1-15_11:15.txt', 'r') as f:
    # game = FishGame(f.readlines())

In [ ]:
agent = QLearningAgent(memories)
agent.train(200)

/Users/Yourui/Documents/Fish/.env/lib/python3.13/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/Users/Yourui/Documents/Fish/nn.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  return torch.FloatTensor(x).to(self.device)
/Users/Yourui/Documents/Fish/nn.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  'ask_card': F.softmax(ask_card),
/Users/Yourui/Documents/Fish/nn.py:85: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting

epoch 0, loss 5.00871, lr 0.001
epoch 1, loss 5.10263, lr 0.001
epoch 2, loss 5.28232, lr 0.001
epoch 3, loss 5.75940, lr 0.001
epoch 4, loss 5.30972, lr 0.001
epoch 5, loss 5.55763, lr 0.001
epoch 6, loss 5.71670, lr 0.0005
epoch 7, loss 4.98524, lr 0.0005
epoch 8, loss 4.75523, lr 0.0005
epoch 9, loss 4.93433, lr 0.0005
epoch 10, loss 4.75982, lr 0.0005
epoch 11, loss 4.89775, lr 0.0005
epoch 12, loss 4.88707, lr 0.0005
epoch 13, loss 4.88698, lr 0.0005
epoch 14, loss 4.65496, lr 0.0005
epoch 15, loss 4.80616, lr 0.0005
epoch 16, loss 4.62904, lr 0.0005
epoch 17, loss 4.73090, lr 0.0005
epoch 18, loss 4.58813, lr 0.0005
epoch 19, loss 4.58007, lr 0.0005
epoch 20, loss 4.37433, lr 0.0005
epoch 21, loss 4.38348, lr 0.0005
epoch 22, loss 4.26815, lr 0.0005
epoch 23, loss 4.33075, lr 0.0005
epoch 24, loss 4.36839, lr 0.0005
epoch 25, loss 4.29413, lr 0.0005
epoch 26, loss 4.34601, lr 0.0005
epoch 27, loss 4.18287, lr 0.0005
epoch 28, loss 4.23045, lr 0.0005
epoch 29, loss 4.30595, lr 0.0